In [7]:
import pandas as pd
import numpy as np

df = pd.read_parquet('../data/open-stax-texts-terms.parquet')

In [59]:
def return_duplicates(dataframe):
    print(f'Remaining Sections: {dataframe.shape[0]}')
    duplicates = (dataframe
                 .loc[dataframe.text.duplicated(keep=False)].book
                 .value_counts()
                 .sort_index()
                )
    print(f'Remaining duplicates: {duplicates.sum()}')
    display(duplicates)
    return dataframe

_ = return_duplicates(df)

Remaining Sections: 4892
Remaining duplicates: 1052


algebra-and-trigonometry-2e                 39
chemistry-2e                                43
chemistry-atoms-first-2e                    43
college-algebra-2e                          40
college-algebra-corequisite-support-2e       3
college-physics-2e                         232
college-physics-ap-courses-2e              232
introduction-business                        3
introductory-business-statistics           122
organizational-behavior                     10
principles-economics-2e                     78
principles-macroeconomics-2e                30
principles-macroeconomics-ap-courses-2e     24
principles-management                       13
principles-microeconomics-2e                76
principles-microeconomics-ap-courses-2e     64
Name: book, dtype: int64

In [60]:
drop_columns = ['college-physics-ap-courses-2e',
                'college-algebra-corequisite-support-2e',
                'principles-macroeconomics-ap-courses-2e',
                'principles-microeconomics-ap-courses-2e',
                'chemistry-atoms-first-2e',
                'introductory-business-statistics',
                'principles-macroeconomics-2e',
                'principles-microeconomics-2e',
                'college-algebra-2e',
                'principles-management',
               ]

deduplicated = df.loc[~df.book.isin(drop_columns)].pipe(return_duplicates)

Remaining Sections: 3855
Remaining duplicates: 0


Series([], Name: book, dtype: int64)

In [62]:
def convert_to_jsonl(dataframe, labels_column='all_term_offsets'):
    (dataframe[['url', 'text', labels_column]]
     .rename(columns={'url':'name',
                      labels_column:'labels'
                     })
     .to_json(f'../assets/openstax_{labels_column}.jsonl',
              orient='records',
              lines=True)
    )

convert_to_jsonl(deduplicated, labels_column='all_term_offsets')

In [63]:
convert_to_jsonl(deduplicated, labels_column='bold_term_offsets')
convert_to_jsonl(deduplicated, labels_column='no_bold_term_offsets')